In [7]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
from transformers import TrainingArguments, Trainer

# Load the custom dataset (assuming it's stored in a CSV file)

import pandas as pd

# Specify the path to your CSV file
csv_file_path = "hausa.csv"

# Specify the text encoding
text_encoding = "utf-8"  # Change this to the appropriate encoding

# Read the CSV file using pandas
df = pd.read_csv(csv_file_path, encoding=text_encoding)


# Split the dataset into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Load the pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = BertTokenizer.from_pretrained(model_name)

# Tokenize and encode the text data
def tokenize_function(examples):
    return tokenizer(examples, padding='max_length', truncation=True)

train_encodings = tokenize_function(train_df['text'].tolist())
val_encodings = tokenize_function(val_df['text'].tolist())




# Convert labels to PyTorch tensors
train_labels = torch.tensor(train_df['label'].map({'negative': 0, 'positive': 1}).values)
val_labels = torch.tensor(val_df['label'].map({'negative': 0, 'positive': 1}).values)

# Create TensorDatasets
train_dataset = TensorDataset(train_encodings, train_labels)
val_dataset = TensorDataset(val_encodings, val_labels)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir='./logs',
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Fine-tune the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()

print(results)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

AttributeError: 